# CoDeepNEAT demo
## CS081 project checkpoint demo
### Harsha Uppli, Alan Zhao, Gabriel Meyer-Lee

The following notebook demonstrates using CoDeepNEAT to solve CIFAR-10

In [11]:
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from math import pi, floor
from random import random
from codeepneat import codeepneat, config, population, chromosome, genome, visualize
import pickle
import numpy as np
import keras

### Problem: CIFAR10 data set

Conveniently, it's also built into Keras, which our CoDeepNEAT imiplementation is built off of.

In [12]:
(x_train_all, y_train_all), (x_test, y_test) = cifar10.load_data()
y_train_all = y_train_all[:,0]
y_test = y_test[:,0]

x_train_all = np.reshape(x_train_all, (x_train_all.shape[0], 32, 32, 3)).astype('float32') / 255
x_test = np.reshape(x_test, (x_test.shape[0], 32, 32, 3)).astype('float32') / 255
y_train_all_one_hot = keras.utils.np_utils.to_categorical(y_train_all)
y_test = keras.utils.np_utils.to_categorical(y_test)

num_categories = 10
category_count = np.zeros(num_categories)

num_training = 42500

x_train = []
y_train = []
x_val = []
y_val = []


index = np.array(range(len(x_train_all)))
np.random.shuffle(index)
x_train_all = x_train_all[index]
y_train_all = y_train_all[index]
y_train_all_one_hot = y_train_all_one_hot[index]
for i in range(len(index)):
    if category_count[y_train_all[i]] < num_training/num_categories:
        x_train.append(x_train_all[i])
        y_train.append(y_train_all_one_hot[i])
        category_count[y_train_all[i]] += 1
    else:
        x_val.append(x_train_all[i])
        y_val.append(y_train_all_one_hot[i])
print(category_count)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)

data = [x_train, y_train, x_val, y_val, x_test, y_test]
print("data shapes")
print("  x train:", x_train.shape)
print("  y train:", y_train.shape)

print("  x val:", x_val.shape)
print("  y val:", y_val.shape)

print("  x test:", x_test.shape)
print("  y test:", y_test.shape)

[ 4250.  4250.  4250.  4250.  4250.  4250.  4250.  4250.  4250.  4250.]
data shapes
  x train: (42500, 32, 32, 3)
  y train: (42500, 10)
  x val: (7500, 32, 32, 3)
  y val: (7500, 10)
  x test: (10000, 32, 32, 3)
  y test: (10000, 10)


### Configuring NEAT

Many of the options and inputs are still handled through the config file. The config file has been shortened considerably as many parameters have been eliminated, although many parameters have also been introduced which could be added.

In [3]:
%%file configCIFAR10
#--- parameters for the robot experiment ---#
[phenotype]
input_nodes         = 32,32,3
output_nodes        = 10
conv                = True
LSTM                = False

[genetic]
max_fitness_threshold = 1

# Human reasoning
pop_size              = 10
prob_addconv          = 0.9
prob_addLSTM          = 0.0
prob_addlayer         = 0.1
prob_mutatelayer      = 0.3
prob_addmodule        = 0.05
prob_switchmodule     = 0.1
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coefficient      = 5.0
disjoint_coefficient    = 3.0
connection_coefficient  = 0.4
size_coefficient        = 0.8

[species]
species_size        = 10
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configCIFAR10


### Fitness

For this demonstration we'll be using supervised learning to train the networks produced by CoDeepNEAT on CIFAR-10 and will use their accuracy after 5 epochs as our fitness. CIFAR-10, like MNIST, is a 10 category classification problem.

In [13]:
def fitness(network, data):
    network.fit(data[0], data[1],  epochs=8)
    loss, acc = network.evaluate(data[2], data[3])
    return acc

### Evolution

Evolution with CoDeepNEAT is slightly different than evolution with NEAT. The main difference is coevolution, where we have two separate populations with a hierarchical relationship evolving together.

In [5]:
def evolve(n, debugging=False):
    if(debugging):
        debug = open("debug.txt", "w")
    else:
        debug = None
    config.load('configCIFAR10')
    # Create 2 separate populations (size is now defined explicitly, but config file can still be used)
    module_pop = population.Population(15, chromosome.ModuleChromo, debug=debug)
    # As the top hierarchical level, the blueprint population needs to be able to see the module population
    blueprint_pop = population.Population(10, chromosome.BlueprintChromo, module_pop, debug=debug)
    # Most of the actual evolving is now handled outside of the population, by CoDeepNEAT
    # Instead of requiring the user to overwrite the evaluation function, CoDeepNEAT evaluates the populations itself,
    # it simply requires a fitness function for the networks it creates passed in as an argument.
    codeepneat.epoch(n, blueprint_pop, module_pop, 25, fitness, data, save_best=True, name='CIFAR10', debug=debug)
    # It will still stop if fitness surpasses the max_fitness_threshold in config file
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(module_pop.stats, name="CIFAR10mod_")
    visualize.plot_stats(blueprint_pop.stats, name="CIFAR10bp_")
    # Visualizes speciation
    #visualize.plot_species(module_pop.species_log, name="NMISTmod_")
    #visualize.plot_species(blueprint_pop.species_log, name="NMISTbp_")

In [6]:
evolve(25, True)

-----Generation 0--------
Network 0
Epoch 1/8
42500/42500 [==============================] - 24s - loss: 1.6378 - acc: 0.4277    
Epoch 2/8
42500/42500 [==============================] - 21s - loss: 1.4748 - acc: 0.4884    
Epoch 3/8
42500/42500 [==============================] - 21s - loss: 1.4261 - acc: 0.5124    
Epoch 4/8
42500/42500 [==============================] - 21s - loss: 1.3894 - acc: 0.5279    
Epoch 5/8
42500/42500 [==============================] - 21s - loss: 1.3580 - acc: 0.5383    
Epoch 6/8
42500/42500 [==============================] - 21s - loss: 1.3307 - acc: 0.5508    
Epoch 7/8
42500/42500 [==============================] - 21s - loss: 1.3009 - acc: 0.5594    
Epoch 8/8
7488/7500 [============================>.] - ETA: 0s
Network 0 Fitness: 0.48
Network 1
Epoch 1/8
42500/42500 [==============================] - 53s - loss: 1.6537 - acc: 0.4236    
Epoch 2/8
42500/42500 [==============================] - 53s - loss: 1.4792 - acc: 0.4870    
Epoch 3/8
42500/42500

42500/42500 [==============================] - 7s - loss: 1.7799 - acc: 0.3604     
Epoch 3/8
42500/42500 [==============================] - 6s - loss: 1.7285 - acc: 0.3772     
Epoch 4/8
42500/42500 [==============================] - 7s - loss: 1.6986 - acc: 0.3894     
Epoch 5/8
42500/42500 [==============================] - 6s - loss: 1.6739 - acc: 0.3979     
Epoch 6/8
42500/42500 [==============================] - 7s - loss: 1.6590 - acc: 0.4048     
Epoch 7/8
42500/42500 [==============================] - 7s - loss: 1.6441 - acc: 0.4083     
Epoch 8/8
7424/7500 [============================>.] - ETA: 0s
Network 19 Fitness: 0.362400000032
Network 20
Epoch 1/8
42500/42500 [==============================] - 29s - loss: 1.6560 - acc: 0.4221    
Epoch 2/8
42500/42500 [==============================] - 28s - loss: 1.4788 - acc: 0.4888    - ETA: 4s - 
Epoch 3/8
42500/42500 [==============================] - 28s - loss: 1.4242 - acc: 0.5121    
Epoch 4/8
42500/42500 [====================

Epoch 1/8
42500/42500 [==============================] - 33s - loss: 1.6546 - acc: 0.4178    
Epoch 2/8
42500/42500 [==============================] - 33s - loss: 1.4813 - acc: 0.4859    - ETA: 0s - loss: 1.480
Epoch 3/8
42500/42500 [==============================] - 33s - loss: 1.4304 - acc: 0.5081    
Epoch 4/8
42500/42500 [==============================] - 33s - loss: 1.3980 - acc: 0.5183    - ETA: 0s - loss: 1.3978 - a
Epoch 5/8
42500/42500 [==============================] - 33s - loss: 1.3768 - acc: 0.5324    - ETA: 3s - los - ETA: 1s -
Epoch 6/8
42500/42500 [==============================] - 33s - loss: 1.3529 - acc: 0.5387    
Epoch 7/8
42500/42500 [==============================] - 33s - loss: 1.3352 - acc: 0.5484    - ETA: 0s - loss: 1.3351 - acc: 0.548
Epoch 8/8
7424/7500 [============================>.] - ETA: 0s
Network 12 Fitness: 0.446400000016
Network 13
Epoch 1/8
42500/42500 [==============================] - 18s - loss: 1.7560 - acc: 0.3879    
Epoch 2/8
42500/42500 [=

42500/42500 [==============================] - 26s - loss: 1.6235 - acc: 0.4266    
Epoch 2/8
42500/42500 [==============================] - 26s - loss: 1.4752 - acc: 0.4865    
Epoch 3/8
42500/42500 [==============================] - 26s - loss: 1.4226 - acc: 0.5068    
Epoch 4/8
42500/42500 [==============================] - 25s - loss: 1.3823 - acc: 0.5230    
Epoch 5/8
42500/42500 [==============================] - 25s - loss: 1.3489 - acc: 0.5372    
Epoch 6/8
42500/42500 [==============================] - 25s - loss: 1.3189 - acc: 0.5489    
Epoch 7/8
42500/42500 [==============================] - 25s - loss: 1.2897 - acc: 0.5616    
Epoch 8/8
7488/7500 [============================>.] - ETA: 0s
Network 5 Fitness: 0.476533333333
Network 6
Epoch 1/8
42500/42500 [==============================] - 17s - loss: 1.6486 - acc: 0.4237    
Epoch 2/8
42500/42500 [==============================] - 17s - loss: 1.4840 - acc: 0.4875    
Epoch 3/8
42500/42500 [==============================] - 

42500/42500 [==============================] - 17s - loss: 1.4943 - acc: 0.4857    
Epoch 3/8
42500/42500 [==============================] - 17s - loss: 1.4408 - acc: 0.5025    
Epoch 4/8
42500/42500 [==============================] - 17s - loss: 1.4030 - acc: 0.5189    
Epoch 5/8
42500/42500 [==============================] - 17s - loss: 1.3728 - acc: 0.5329    
Epoch 6/8
42500/42500 [==============================] - 17s - loss: 1.3486 - acc: 0.5394    
Epoch 7/8
42500/42500 [==============================] - 17s - loss: 1.3237 - acc: 0.5504    
Epoch 8/8
7500/7500 [==============================] - 2s     

Network 24 Fitness: 0.4872
-----Modules-----------

 ****** Generation 2 ****** 

Population's average fitness: 0.43269 stdev: 0.03133
Best fitness: 0.4850000000 - size: 1 - species 2 - id 33
Species length: 5 totalizing 15 individuals
Species ID       : [1, 2, 3, 7, 8]
Each species size: [1, 4, 1, 5, 4]
Amount to spawn  : [3, 3, 3, 3, 3]
Species age      : [2, 2, 2, 0, 0]
Specie

...

-----Generation 8--------
Network 0
Epoch 1/8
42500/42500 [==============================] - 20s - loss: 1.4779 - acc: 0.4765    
Epoch 2/8
42500/42500 [==============================] - 16s - loss: 1.0593 - acc: 0.6332    
Epoch 3/8
42500/42500 [==============================] - 16s - loss: 0.8840 - acc: 0.6965    
Epoch 4/8
42500/42500 [==============================] - 16s - loss: 0.7486 - acc: 0.7449    
Epoch 5/8
42500/42500 [==============================] - 16s - loss: 0.6378 - acc: 0.7842    
Epoch 6/8
42500/42500 [==============================] - 16s - loss: 0.5329 - acc: 0.8219    
Epoch 7/8
42500/42500 [==============================] - 16s - loss: 0.4410 - acc: 0.8542    
Epoch 8/8
7424/7500 [============================>.] - ETA: 0s
Network 0 Fitness: 0.608666666698
Network 1
Epoch 1/8
42500/42500 [==============================] - 58s - loss: 1.6039 - acc: 0.4266    
Epoch 2/8
42500/42500 [==============================] - 53s - loss: 1.1976 - acc: 0.5781    
Epoch 3/8
4

42500/42500 [==============================] - 58s - loss: 1.2747 - acc: 0.5478    
Epoch 7/8
42500/42500 [==============================] - 58s - loss: 1.2766 - acc: 0.5489    
Epoch 8/8
7456/7500 [============================>.] - ETA: 0s
Network 9 Fitness: 0.473466666667
Network 10
Epoch 1/8
42500/42500 [==============================] - 19s - loss: 1.4333 - acc: 0.4964    
Epoch 2/8
42500/42500 [==============================] - 14s - loss: 1.0525 - acc: 0.6365    
Epoch 3/8
42500/42500 [==============================] - 14s - loss: 0.8981 - acc: 0.6952    
Epoch 4/8
42500/42500 [==============================] - 14s - loss: 0.7921 - acc: 0.7313    
Epoch 5/8
42500/42500 [==============================] - 14s - loss: 0.7052 - acc: 0.7614    
Epoch 6/8
42500/42500 [==============================] - 14s - loss: 0.6220 - acc: 0.7926    
Epoch 7/8
42500/42500 [==============================] - 14s - loss: 0.5488 - acc: 0.8178    
Epoch 8/8
7392/7500 [============================>.] - E

42500/42500 [==============================] - 14s - loss: 0.7047 - acc: 0.7639    
Epoch 6/8
42500/42500 [==============================] - 15s - loss: 0.6222 - acc: 0.7956    
Epoch 7/8
42500/42500 [==============================] - 14s - loss: 0.5406 - acc: 0.8213    
Epoch 8/8
7488/7500 [============================>.] - ETA: 0s
Network 2 Fitness: 0.642800000032
Network 3
Epoch 1/8
42500/42500 [==============================] - 35s - loss: 1.4772 - acc: 0.4662    - ETA: 1s - loss: 1.48
Epoch 2/8
42500/42500 [==============================] - 31s - loss: 1.1802 - acc: 0.5789    
Epoch 3/8
42500/42500 [==============================] - 31s - loss: 1.0387 - acc: 0.6307    - ETA: 4s - E
Epoch 4/8
42500/42500 [==============================] - 31s - loss: 0.9247 - acc: 0.6745    
Epoch 5/8
42500/42500 [==============================] - 31s - loss: 0.8366 - acc: 0.7058    
Epoch 6/8
42500/42500 [==============================] - 31s - loss: 0.7510 - acc: 0.7367    
Epoch 7/8
42500/42500 

Network 12
Epoch 1/8
42500/42500 [==============================] - 23s - loss: 1.4466 - acc: 0.4902    
Epoch 2/8
42500/42500 [==============================] - 17s - loss: 1.0355 - acc: 0.6418    
Epoch 3/8
42500/42500 [==============================] - 17s - loss: 0.8587 - acc: 0.7050    
Epoch 4/8
42500/42500 [==============================] - 17s - loss: 0.7299 - acc: 0.7527    
Epoch 5/8
42500/42500 [==============================] - 17s - loss: 0.6101 - acc: 0.7938    
Epoch 6/8
42500/42500 [==============================] - 17s - loss: 0.5044 - acc: 0.8319    
Epoch 7/8
42500/42500 [==============================] - 17s - loss: 0.4107 - acc: 0.8661    
Epoch 8/8
7424/7500 [============================>.] - ETA: 0s
Network 12 Fitness: 0.618666666698
Network 13
Epoch 1/8
42500/42500 [==============================] - 93s - loss: 1.6505 - acc: 0.4129    - - ETA: 2s - loss: 1.66 - ETA: 1s - loss: 1.6565 - acc: 0.4 - ETA: 1s - loss: 1.6556 - acc: 0.411 - ETA: 1s - loss: 1.
Epoch 2/8

ResourceExhaustedError: OOM when allocating tensor with shape[32,256,32,32]
	 [[Node: model_1036/conv2d_6/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](model_1036/conv2d_14/Relu, conv2d_6_288/kernel/read)]]
	 [[Node: loss_242/mul/_23485 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1985_loss_242/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'model_1036/conv2d_6/convolution', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/scratch/knerr/venvs/cs81venv/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.la